In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re
import string
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import os
nltk.download('stopwords')
nltk.download('wordnet')
random_state=24

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chaud\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chaud\AppData\Roaming\nltk_data...


In [3]:
!git init

Initialized empty Git repository in C:/Users/chaud/OneDrive/Desktop/Applied ML/Assignment_2/.git/


In [4]:
!dvc init

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [6]:
# 1. Load the data
raw_messages = pd.read_csv('SMSSpamCollection', sep='\t', quoting=csv.QUOTE_NONE,names=["label", "message"])

In [7]:
raw_messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
# Save the raw data
raw_messages.to_csv("raw_data.csv", index=False)

# Track the raw data using DVC
!dvc add raw_data.csv
!git add raw_data.csv.dvc
!git commit -m "Added raw data"


To track the changes with git, run:

	git add raw_data.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


\u280b Checking graph



On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	.ipynb_checkpoints/
	SMSSpamCollection
	prepare.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [10]:
# Define stop words and stemmer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [11]:
def preprocess_text(text):
    """
    Clean and preprocess a single text message.
    """
    text = text.lower() # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
    tokens = word_tokenize(text) # Tokenize words
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

def preprocess_data(data):
    """
    Preprocess the entire dataset.
    """
    data['message'] = data['message'].apply(preprocess_text)
    # Encode labels: spam -> 1, ham -> 0
    data['label'] = data['label'].map({'spam': 1, 'ham': 0})
    return data

In [14]:
data = preprocess_data(raw_messages)
data.head()

,label,message
0,0,go jurong point crazy available bugis n great ...
1,0,ok lar joking wif u oni
2,1,free entry wkly comp win fa cup final tkts st ...
3,0,u dun say early hor u c already say
4,0,nah dont think go usf life around though


In [15]:
X = data['message']
y = data['label']

In [16]:
def split_and_save_data(X, y, label_column="label", test_size=0.2, val_size=0.1, random_state=24):
    """
    Split the data into train, validation, and test sets, and save them as CSV files.
    """
    #Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    
    #Train-validation split
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=val_size, random_state=random_state
    )
    
    train_df = pd.DataFrame(X_train)
    train_df['label'] = y_train

    test_df = pd.DataFrame(X_test)
    test_df['label'] = y_test

    val_df = pd.DataFrame(X_val)
    val_df['label'] = y_val

    train_df.to_csv(f"./train.csv", index=False)
    test_df.to_csv(f"./test.csv", index=False)
    val_df.to_csv(f"./val.csv", index=False)

    print("Data splits saved successfully!")


In [17]:
split_and_save_data(X, y, label_column="label", test_size=0.2, val_size=0.1, random_state=24)

Data splits saved successfully!


In [19]:
# Track the new version with DVC
!dvc add train.csv val.csv test.csv
!git add train.csv.dvc val.csv.dvc test.csv.dvc
!git commit -m "Version 1 of train/validation/test split"


To track the changes with git, run:

	git add val.csv.dvc train.csv.dvc test.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


\u280b Checking graph



On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	.ipynb_checkpoints/
	SMSSpamCollection
	prepare.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [20]:
# Perform new split with a different random seed
split_and_save_data(X, y, label_column="label", test_size=0.2, val_size=0.1, random_state=80)

Data splits saved successfully!


In [21]:
# Track the new version with DVC
!dvc add train.csv val.csv test.csv
!git add train.csv.dvc val.csv.dvc test.csv.dvc
!git commit -m "Version 2 of train/validation/test split"


To track the changes with git, run:

	git add test.csv.dvc val.csv.dvc train.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


\u280b Checking graph



[master f6dd6c3] Version 2 of train/validation/test split
 3 files changed, 6 insertions(+), 6 deletions(-)


In [22]:
# Load and print class distributions
train = pd.read_csv("train.csv")
val = pd.read_csv("val.csv")
test = pd.read_csv("test.csv")

print("Version 1: class distribution:")
print("Train:\n", train["label"].value_counts())
print("Validation:\n", val["label"].value_counts())
print("Test:\n", test["label"].value_counts())

Version 1: class distribution:
Train:
 label
0    3487
1     526
Name: count, dtype: int64
Validation:
 label
0    372
1     74
Name: count, dtype: int64
Test:
 label
0    968
1    147
Name: count, dtype: int64


In [23]:
!dvc checkout

In [24]:
# Load and print class distributions
train = pd.read_csv("train.csv")
val = pd.read_csv("val.csv")
test = pd.read_csv("test.csv")

print("Version 2: class distribution:")
print("Train:\n", train["label"].value_counts())
print("Validation:\n", val["label"].value_counts())
print("Test:\n", test["label"].value_counts())

Version 2: class distribution:
Train:
 label
0    3487
1     526
Name: count, dtype: int64
Validation:
 label
0    372
1     74
Name: count, dtype: int64
Test:
 label
0    968
1    147
Name: count, dtype: int64
